# modeling4-nli-binaryclassifier-Dataset
- CustomDataset Class for Binray input 
    1) q&a Fid Format
    2) NLI Format
- Metric

In [2]:
import json
import os
import argparse
import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

import json
from copy import deepcopy
from pprint import pprint
from util import utils
# from transformers.models.longformer impor LongformerForSequenceClassification, LongformerSequenceClassifierOutput
# from transformers.modeling_outputs import LongformerSequenceClassifierOutput
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
# parser.parse_args_into_dataclasses([])
# args = ["--foo", "1", "--baz", "quux", "--bar", "0.5"]
args = ["--model_name_or_path", 'roberta-large', '--output_dir', './']
# (example,) = parser.parse_args_into_dataclasses(args, look_for_args_file=False)
# args = parser.parse_args(args=[])
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
# print(model_args.model_name_or_path)

In [4]:
print(f'model_args : \n ')
pprint(model_args)
print(f'data_args : \n ')
pprint(data_args)
print(f'training_args : \n ')
pprint(data_args)

model_args : 
 
ModelArguments(model_architecture='roberta-large', model_name_or_path='roberta-large', git_tag='v1.1', config_name=None, tokenizer_name=None, max_seq_length=200)
data_args : 
 
DataTrainingArguments(data='NQ-DEV-DPR/5-fold/1', train_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', eval_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', num_labels=2, overwrite_cache=False, pad_to_max_length=False)
training_args : 
 
DataTrainingArguments(data='NQ-DEV-DPR/5-fold/1', train_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', eval_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', num_labels=2, overwrite_cache=False, pad_to_max_length=False)


In [5]:
parser = argparse.ArgumentParser()
MY_DICT = {'key' : 'value'}

In [6]:
parser.add_argument('-s', '--seed', type=int, default=42, help='random seed (default : 42)')
parser.add_argument('-m', '--model', type=str, default='BaseModel', help='model type (default:BaseModel)')
parser.add_argument('--train-dataset', type=str, default='NQ', help='train dataset (default:Natural Question)')
parser.add_argument('--eval_dataset', type=str, default='NQ', help='eval dataset (default:Natural Question)')
parser.add_argument('--output_dir', type=str, default='output directory', help='output directory')
# parser.add_argument('--cache_directory', type = str, default = '', help='default transformer cahce directory')

_StoreAction(option_strings=['--output_dir'], dest='output_dir', nargs=None, const=None, default='output directory', type=<class 'str'>, choices=None, help='output directory', metavar=None)

In [7]:
args = parser.parse_args([])

In [8]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [9]:
# print(args)

In [10]:
seed_everything(args.seed)

In [10]:
# config file are updated when executing .save_pretrained('./')
# model architecture depends on model_name
model = AutoModelForSequenceClassification.from_pretrained('roberta-large', num_labels=2)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [11]:
# TEST1
# model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-large-4096')
# model = AutoModel.from_pretrained('allenai/longformer-large-4096')
# type(model)

# TEST2
# Custom LongformerForSequenceClassification
# class CustomSequenceClassification(nn.Module):
#     def __init__(self, model_name, num_labels):
#         super(CustomSequenceClassification, self).__init__()
#         self.num_labels = num_labels
#         self.model_name = model_name
        
#         # Base Model Configuration
#         self.model = AutoModel.from_pretrained(self.model_name, add_pooling_layer = False)
#         self.config = self.model.config
        
#         # Custom Layers for Sequence Classification
#         self.dense = nn.Linear(self.config.hidden_size, self.config.hidden_size)
#         self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
#         self.out_proj = nn.Linear(self.config.hidden_size, self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(
#             input_ids=input_ids, 
#             attention_mask=attention_mask,
#         )
        
#         # outputs[0] -> last hidden layer 
#         sequence_output = outputs[0]
        
#         # [batch, sequence, hidden] -> [:, 0, :] -> hidden state of <s> or [CLS] token 
#         hidden_states = hidden_states[:, 0, :]
#         hidden_states = self.dropout(hidden_states)
#         hidden_states = self.dense(hidden_states)
#         hidden_states = torch.tanh(hidden_states)
#         hidden_states = self.dropout(hidden_states)
#         logits = self.out_proj(hidden_states)
        
#         loss = None
#         if labels is not None:
#             # single_label_classification
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), lablels.view(-1))
            
#         return SequenceClassifierOutput(
#             loss=loss, 
#             logits=logits, 
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#             global_attentions=outputs.gloabl_attentions
#         )

# TEST3
# class CustomSequenceClassification(AutoModelForSequenceClassification):
#     def __init__(self, model_name, num_labels):
#         super().__init__()
#         self.model_name = model_name
#         self.num_labels = num_labels
#         self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
#         return outputs

## nq

In [12]:
# nq_test_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-dev-multikilt.json'
# nq_dpr_dev = open_json(nq_test_file)

In [13]:
# nq_train_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-train-multikilt.json'
# nq_dpr_train = open_json(nq_train_file)

In [14]:
# print(len(nq_dpr_dev))
# print(len(nq_dpr_train))

# Binray data 

In [11]:
binary_train_file = '/data/philhoon-relevance/binary-classification\
/NQ-DEV-DPR/5-fold/1/binary_data/\
binary_ex_ctx100id_split_train_1.json'

binary_dev_file = '/data/philhoon-relevance/binary-classification\
/NQ-DEV-DPR/5-fold/1/binary_data/\
binary_ex_ctx100id_split_dev_1.json'

# binary_dev_file = 'binary_ex_ctx100id_split_dev_1.json'
max_length = 200

In [12]:
binary_train_file

'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1.json'

In [13]:
binary_dev_file

'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_dev_1.json'

In [14]:
binary_train = utils.open_json(binary_train_file)
binary_dev = utils.open_json(binary_dev_file)

In [15]:
print(len(binary_train))
print(len(binary_dev))

681487
169978


In [16]:
model_name = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
type(binary_train)

list

In [18]:
binary_train[0]

{'id': 1,
 'question': 'how many pages is invisible man by ralph ellison',
 'ctx': {'id': '533920',
  'title': 'Ralph Ellison',
  'text': 'he was unsatisfied with the book. Ellison ultimately wrote more than 2,000 pages of this second novel but never finished it. Ellison died on April 16, 1994 of pancreatic cancer and was interred in a crypt at Trinity Church Cemetery in the Washington Heights neighborhood of Upper Manhattan. He was survived by his second wife, Fanny Ellison (November 27, 1911 – November 19, 2005). "Invisible Man" won the 1953 US National Book Award for Fiction. The award was his ticket into the American literary establishment. He eventually was admitted to the American Academy of Arts and Letters, received two President\'s'},
 'em': '0'}

In [19]:
pprint(binary_train[0])

{'ctx': {'id': '533920',
         'text': 'he was unsatisfied with the book. Ellison ultimately wrote '
                 'more than 2,000 pages of this second novel but never '
                 'finished it. Ellison died on April 16, 1994 of pancreatic '
                 'cancer and was interred in a crypt at Trinity Church '
                 'Cemetery in the Washington Heights neighborhood of Upper '
                 'Manhattan. He was survived by his second wife, Fanny Ellison '
                 '(November 27, 1911 – November 19, 2005). "Invisible Man" won '
                 'the 1953 US National Book Award for Fiction. The award was '
                 'his ticket into the American literary establishment. He '
                 'eventually was admitted to the American Academy of Arts and '
                 "Letters, received two President's",
         'title': 'Ralph Ellison'},
 'em': '0',
 'id': 1,
 'question': 'how many pages is invisible man by ralph ellison'}


In [ ]:
def preprocess_function(examples):
        # Tokenize the texts
        texts = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*texts, padding=padding, max_length=args.max_length, truncation=True)

        if "label" in examples:
            if label_to_id is not None:
                # Map labels to IDs (not necessary for GLUE tasks)
                result["labels"] = [label_to_id[l] for l in examples["label"]]
            else:
                # In all cases, rename the column to labels because the model will expect that.
                result["labels"] = examples["label"]
        return result

In [20]:
class BinarySentenceDataset(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length
    
    def __len__(self):
        return len(self.instances)
    
    def __getitem__(self, idx):
        text1_ = 'question: ' + self.instances[idx]['question']
        
        text2_ = 'title: ' + self.instances[idx]['ctx']['title'] + \
                    ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            text1_, text2_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [21]:
train_dataset_testing = BinarySentenceDataset(binary_train,
    tokenizer,
    max_length,
    False)

In [22]:
print(train_dataset_testing[0])

{'input_ids': [0, 40018, 35, 141, 171, 6052, 16, 20731, 313, 30, 910, 40812, 28041, 4060, 2, 2, 14691, 35, 10594, 21930, 5377, 4832, 37, 21, 36010, 2550, 19, 5, 1040, 4, 21930, 3284, 875, 55, 87, 132, 6, 151, 6052, 9, 42, 200, 5808, 53, 393, 1550, 24, 4, 21930, 962, 15, 587, 545, 6, 8148, 9, 30737, 636, 1668, 8, 21, 3222, 2050, 11, 10, 35867, 23, 13544, 2197, 12472, 11, 5, 663, 9754, 3757, 9, 11851, 6562, 4, 91, 21, 5601, 30, 39, 200, 1141, 6, 274, 13749, 21930, 36, 21206, 974, 6, 39711, 126, 759, 753, 6, 4013, 322, 22, 1121, 42152, 1554, 113, 351, 5, 23443, 382, 496, 5972, 3683, 13, 35320, 4, 20, 2354, 21, 39, 3682, 88, 5, 470, 17205, 7147, 4, 91, 2140, 21, 2641, 7, 5, 470, 3536, 9, 4455, 8, 23937, 6, 829, 80, 270, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [41]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_dataset_testing[1]['input_ids']))

'<s>question: how many pages is invisible man by ralph ellison</s></s>title: Ralph Ellison context : Ralph Ellison Ralph Waldo Ellison (March 1, 1913 – April 16, 1994) was an American novelist, literary critic, and scholar. Ellison is best known for his novel "Invisible Man", which won the National Book Award in 1953. He also wrote "Shadow and Act" (1964), a collection of political, social and critical essays, and "Going to the Territory" (1986). For "The New York Times", the best of these essays in addition to the novel put him "among the gods of America\'s literary Parnassus." A posthumous novel, "Juneteenth", was published after being assembled from voluminous notes he left upon his death. Ralph</s>'

In [43]:
tokenizer.cls_token

'<s>'

In [44]:
class BinaryCustomDatasetShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [45]:
train_dataset = BinaryCustomDatasetShuffle(binary_train,
    tokenizer,
    max_length,
    False)

In [46]:
train_dataset.instances[0]['em']

'0'

In [47]:
print(train_dataset[1])

{'input_ids': [0, 40018, 35, 141, 171, 6052, 16, 20731, 313, 30, 910, 40812, 28041, 4060, 1270, 35, 10594, 21930, 5377, 4832, 10594, 21930, 10594, 15371, 139, 21930, 36, 11770, 112, 6, 38220, 126, 587, 545, 6, 8148, 43, 21, 41, 470, 29613, 6, 17205, 7745, 6, 8, 20875, 4, 21930, 16, 275, 684, 13, 39, 5808, 22, 1121, 42152, 1554, 1297, 61, 351, 5, 496, 5972, 3683, 11, 23443, 4, 91, 67, 875, 22, 45418, 8, 1783, 113, 36, 45629, 238, 10, 2783, 9, 559, 6, 592, 8, 2008, 27616, 6, 8, 22, 27524, 7, 5, 23463, 113, 36, 43479, 322, 286, 22, 133, 188, 469, 1513, 1297, 5, 275, 9, 209, 27616, 11, 1285, 7, 5, 5808, 342, 123, 22, 31636, 5, 25779, 9, 730, 18, 17205, 221, 4422, 2401, 687, 72, 83, 618, 18257, 1827, 5808, 6, 22, 31036, 45248, 1297, 21, 1027, 71, 145, 14525, 31, 13103, 18140, 1827, 2775, 37, 314, 2115, 39, 744, 4, 10594, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [25]:
pprint(binary_train[0])

{'ctx': {'id': '533920',
         'text': 'he was unsatisfied with the book. Ellison ultimately wrote '
                 'more than 2,000 pages of this second novel but never '
                 'finished it. Ellison died on April 16, 1994 of pancreatic '
                 'cancer and was interred in a crypt at Trinity Church '
                 'Cemetery in the Washington Heights neighborhood of Upper '
                 'Manhattan. He was survived by his second wife, Fanny Ellison '
                 '(November 27, 1911 – November 19, 2005). "Invisible Man" won '
                 'the 1953 US National Book Award for Fiction. The award was '
                 'his ticket into the American literary establishment. He '
                 'eventually was admitted to the American Academy of Arts and '
                 "Letters, received two President's",
         'title': 'Ralph Ellison'},
 'em': '0',
 'id': 1,
 'question': 'how many pages is invisible man by ralph ellison'}


In [26]:
binary_train[0]

{'id': 1,
 'question': 'how many pages is invisible man by ralph ellison',
 'ctx': {'id': '533920',
  'title': 'Ralph Ellison',
  'text': 'he was unsatisfied with the book. Ellison ultimately wrote more than 2,000 pages of this second novel but never finished it. Ellison died on April 16, 1994 of pancreatic cancer and was interred in a crypt at Trinity Church Cemetery in the Washington Heights neighborhood of Upper Manhattan. He was survived by his second wife, Fanny Ellison (November 27, 1911 – November 19, 2005). "Invisible Man" won the 1953 US National Book Award for Fiction. The award was his ticket into the American literary establishment. He eventually was admitted to the American Academy of Arts and Letters, received two President\'s'},
 'em': '0'}

In [31]:
instances = binary_train
idx = 0
sep_token = tokenizer.sep_token
max_length = 200

In [32]:
instances[0]

{'id': 1,
 'question': 'how many pages is invisible man by ralph ellison',
 'ctx': {'id': '533920',
  'title': 'Ralph Ellison',
  'text': 'he was unsatisfied with the book. Ellison ultimately wrote more than 2,000 pages of this second novel but never finished it. Ellison died on April 16, 1994 of pancreatic cancer and was interred in a crypt at Trinity Church Cemetery in the Washington Heights neighborhood of Upper Manhattan. He was survived by his second wife, Fanny Ellison (November 27, 1911 – November 19, 2005). "Invisible Man" won the 1953 US National Book Award for Fiction. The award was his ticket into the American literary establishment. He eventually was admitted to the American Academy of Arts and Letters, received two President\'s'},
 'em': '0'}

In [33]:
input_ = 'question: ' + instances[idx]['question'] + \
        ' title: ' + instances[idx]['ctx']['title'] + \
        ' context : ' + instances[idx]['ctx']['text']

In [34]:
output = tokenizer(
            input_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=max_length)

In [35]:
input_

'question: how many pages is invisible man by ralph ellison title: Ralph Ellison context : he was unsatisfied with the book. Ellison ultimately wrote more than 2,000 pages of this second novel but never finished it. Ellison died on April 16, 1994 of pancreatic cancer and was interred in a crypt at Trinity Church Cemetery in the Washington Heights neighborhood of Upper Manhattan. He was survived by his second wife, Fanny Ellison (November 27, 1911 – November 19, 2005). "Invisible Man" won the 1953 US National Book Award for Fiction. The award was his ticket into the American literary establishment. He eventually was admitted to the American Academy of Arts and Letters, received two President\'s'

In [36]:
print(len(output['input_ids']))
print(output['input_ids'])

143
[0, 40018, 35, 141, 171, 6052, 16, 20731, 313, 30, 910, 40812, 28041, 4060, 1270, 35, 10594, 21930, 5377, 4832, 37, 21, 36010, 2550, 19, 5, 1040, 4, 21930, 3284, 875, 55, 87, 132, 6, 151, 6052, 9, 42, 200, 5808, 53, 393, 1550, 24, 4, 21930, 962, 15, 587, 545, 6, 8148, 9, 30737, 636, 1668, 8, 21, 3222, 2050, 11, 10, 35867, 23, 13544, 2197, 12472, 11, 5, 663, 9754, 3757, 9, 11851, 6562, 4, 91, 21, 5601, 30, 39, 200, 1141, 6, 274, 13749, 21930, 36, 21206, 974, 6, 39711, 126, 759, 753, 6, 4013, 322, 22, 1121, 42152, 1554, 113, 351, 5, 23443, 382, 496, 5972, 3683, 13, 35320, 4, 20, 2354, 21, 39, 3682, 88, 5, 470, 17205, 7147, 4, 91, 2140, 21, 2641, 7, 5, 470, 3536, 9, 4455, 8, 23937, 6, 829, 80, 270, 18, 2]


In [37]:
print(tokenizer.convert_ids_to_tokens(output['input_ids']))
print(len(tokenizer.convert_ids_to_tokens(output['input_ids'])))

['<s>', 'question', ':', 'Ġhow', 'Ġmany', 'Ġpages', 'Ġis', 'Ġinvisible', 'Ġman', 'Ġby', 'Ġr', 'alph', 'Ġell', 'ison', 'Ġtitle', ':', 'ĠRalph', 'ĠEllison', 'Ġcontext', 'Ġ:', 'Ġhe', 'Ġwas', 'Ġunsatisf', 'ied', 'Ġwith', 'Ġthe', 'Ġbook', '.', 'ĠEllison', 'Ġultimately', 'Ġwrote', 'Ġmore', 'Ġthan', 'Ġ2', ',', '000', 'Ġpages', 'Ġof', 'Ġthis', 'Ġsecond', 'Ġnovel', 'Ġbut', 'Ġnever', 'Ġfinished', 'Ġit', '.', 'ĠEllison', 'Ġdied', 'Ġon', 'ĠApril', 'Ġ16', ',', 'Ġ1994', 'Ġof', 'Ġpancreat', 'ic', 'Ġcancer', 'Ġand', 'Ġwas', 'Ġinter', 'red', 'Ġin', 'Ġa', 'Ġcrypt', 'Ġat', 'ĠTrinity', 'ĠChurch', 'ĠCemetery', 'Ġin', 'Ġthe', 'ĠWashington', 'ĠHeights', 'Ġneighborhood', 'Ġof', 'ĠUpper', 'ĠManhattan', '.', 'ĠHe', 'Ġwas', 'Ġsurvived', 'Ġby', 'Ġhis', 'Ġsecond', 'Ġwife', ',', 'ĠF', 'anny', 'ĠEllison', 'Ġ(', 'November', 'Ġ27', ',', 'Ġ1911', 'ĠâĢĵ', 'ĠNovember', 'Ġ19', ',', 'Ġ2005', ').', 'Ġ"', 'In', 'visible', 'ĠMan', '"', 'Ġwon', 'Ġthe', 'Ġ1953', 'ĠUS', 'ĠNational', 'ĠBook', 'ĠAward', 'Ġfor', 'ĠFiction', '.', 'Ġ

In [38]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(output['input_ids']))

'<s>question: how many pages is invisible man by ralph ellison title: Ralph Ellison context : he was unsatisfied with the book. Ellison ultimately wrote more than 2,000 pages of this second novel but never finished it. Ellison died on April 16, 1994 of pancreatic cancer and was interred in a crypt at Trinity Church Cemetery in the Washington Heights neighborhood of Upper Manhattan. He was survived by his second wife, Fanny Ellison (November 27, 1911 – November 19, 2005). "Invisible Man" won the 1953 US National Book Award for Fiction. The award was his ticket into the American literary establishment. He eventually was admitted to the American Academy of Arts and Letters, received two President\'s</s>'

In [ ]:
from tqdm import tqdm

In [ ]:
cnt = 0
for idx, instance in tqdm(enumerate(instances)):
    input_ = 'question: ' + instance['question'] + \
        ' title: ' + instance['ctx']['title'] + \
        ' context : ' + instance['ctx']['text']
    
    output = tokenizer(
            input_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=max_length)
    
    if len(output['input_ids']) > max_length:
        cnt += 1
print(cnt)

In [ ]:
binary_train[0]

In [ ]:
'question: ' + binary_train[0]['question'] + ' title: ' + binary_train[0]['ctx']['title'] + ' context : ' + binary_train[0]['ctx']['text']

In [ ]:
binary_train[0]['ctx']['title']

In [ ]:
binary_train[0]['ctx']['text']

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [ ]:
data_collator = DataCollatorWithPadding(
        tokenizer, 
        pad_to_multiple_of=8,
    )

In [ ]:
# Trainer implement dataloader inside
# data_loader = DataLoader(
#     dataset, 
#     batch_size=8, 
#     shuffle=True, 
#     sampler=None,
#     batch_sampler=None, 
#     num_workers=1, 
#     collate_fn=data_collator,
#     pin_memory=False, 
#     drop_last=False, 
#     timeout=0,
#     worker_init_fn=None)

# Metric

In [ ]:
from datasets import load_metric

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load("xnli")

In [ ]:
metric

In [ ]:
metric3 = evaluate.load("f1")

In [ ]:
metric3